In [16]:
import ifcopenshell
import ifcopenshell.geom as geom

import ifcopenshell.api.alignment

file = ifcopenshell.open("F:/IFC-Rail-Unit-Test-Reference-Code/alignment_testset/IFC-WithGeneratedGeometry/GENERATED__HorizontalAlignment_BlossCurve_100.0_300_1000_1_Meter.ifc")

inst = model.by_type('IfcCompositeCurve')[0]
print(inst)

# Create a IfcBuildingElementProxy
# Set the ObjectPlacement using an IfcLinearPlacement where the IfcPointByDistanceExpression references the composite curve
# Create a new IfcCompositeCurve with an IfcCurveSegment and IfcThirdOrderPolynomialSpiral that is NOT a Bloss curve (use a fancy spiral)
# Create an IfcSweptDiskSolid using the crazy curve
# Save the model into a new file

A0 = 1
A1 = 2
A2 = 3
A3 = 4

parent_curve = file.createIfcThirdOrderPolynomialSpiral(
    Position=file.createIfcAxis2Placement2D(
        Location=file.createIfcCartesianPoint((0.0, 0.0)), RefDirection=file.createIfcDirection((1.0, 0.0))
    ),
    CubicTerm=A3,
    QuadraticTerm=A2 if A2 != 0.0 else None,
    LinearTerm=A1 if A1 != 0.0 else None,
    ConstantTerm=A0 if A0 != 0.0 else None,
)

curve_segment = file.create_entity(
    type="IfcCurveSegment",
    Transition="DISCONTINUOUS",
    Placement=file.createIfcAxis2Placement2D(
        Location=file.createIfcCartesianPoint((0.0,0.0)),
        RefDirection=file.createIfcDirection((1.0,0.0)),
    ),
    SegmentStart=file.createIfcLengthMeasure(-50.0),
    SegmentLength=file.createIfcLengthMeasure(50.0),
    ParentCurve=parent_curve,
)

composite_curve = file.createIfcCompositeCurve(Segments=[curve_segment,])

object_placement = file.createIfcLinearPlacement(
                    RelativePlacement=file.createIfcAxis2PlacementLinear(
                       Location=file.createIfcPointByDistanceExpression(
                       DistanceAlong=file.createIfcLengthMeasure(50.0),
                       OffsetLateral=None,
                       OffsetVertical=None,
                       OffsetLongitudinal=None,
                       BasisCurve=inst,
                   )
               ),
            CartesianPosition=None,
        )

swept_disk_solid = file.createIfcSweptDiskSolid(Directrix=composite_curve,Radius=0.05)

context = ifcopenshell.util.representation.get_context(file,"Model","Body","MODEL_VIEW")
if context == None:
    geometric_representation_context = ifcopenshell.api.context.add_context(file, context_type="Model")
    context = ifcopenshell.api.context.add_context(
        file,
        context_type="Model",
        context_identifier="Body",
        target_view="MODEL_VIEW",
        parent=geometric_representation_context,
    )    
    
shape_representation = file.createIfcShapeRepresentation(ContextOfItems=context,RepresentationIdentifier="Body",RepresentationType="AdvancedSweptSolid",Items=(swept_disk_solid,))

proxy = file.createIfcBuildingElementProxy(
    GlobalId=ifcopenshell.guid.new(),
    ObjectPlacement=object_placement
)

ifcopenshell.api.geometry.assign_representation(file,proxy,shape_representation)

file.write("C:\\Users\\bricer\\OneDrive - Washington State Department of Transportation\\Desktop\\GenericSpiral.ifc")


#35=IfcCompositeCurve((#36,#48),.F.)
